# Checking Retrosheet and Lahman Data Consistency

An incredible amount of work has gone into making the open-source Retrosheet and Lahman data sets as accurate as possible, but they are not perfectly accurate.

Prior to performing any data analysis, it is necessary to check the accuracy of the data.  The limitations of the data will define the limitations of the analysis.

Performing data consistency tests is also a great way to find errors in the data processing pipeline as well as to learn about the data.

A brief summary of the wrangled data is provided below: [MLB Data Summary](#MLB-Data-Summary)

There are three types of data consistency tests that will be performed
* Retrosheet vs Lahman
  * batting, pitching and fielding data will be aggregated to the same level and compared
* Detailed data vs Summarized data
  * Retrosheet: aggregate daily data to team data (per game) and compare
    * compares cwdaily parser results with cwgame parser results
  * Lahman: aggregate individual data to team data and compare
* Batting vs Pitching Allowed
  * Retrosheet: use each player's daily data
  * Lahman: use each player's seasonal data
  
Prior to performing the data consistency tests, it is necessary to verify that the fields which should uniquely identify a record, actually do.

Unusual data can be spot checked against Baseball-Reference to verify it.

The pytest test suite supplied in this repo automatically runs a superset of the tests performed in this notebook.  The purpose of this notebook is to explain these data consistency tests in more detail to help understand the data.

All data between 1974 and 2019 inclusive will be used.  Retrosheet has all play-by-play data over this time period.

## Data Tests
[Primary and Foreign Key Tests](#Primary-and-Foreign-Key-Tests)  

[Retrosheet Batting vs Lahman Batting Players](#Retrosheet-Batting-vs-Lahman-Batting-Players)  
[Retrosheet Batting vs Lahman Batting Stats](#Retrosheet-Batting-vs-Lahman-Batting-Stats)  
[Retrosheet Pitching vs Lahman Pitching Players](#Retrosheet-Pitching-vs-Lahman-Pitching-Players)  
[Retrosheet Pitching vs Lahman Pitching Stats](#Retrosheet-Pitching-vs-Lahman-Pitching-Stats)  
[Retrosheet Fielding vs Lahman Fielding Players](#Retrosheet-Fielding-vs-Lahman-Fielding-Players)  
[Retrosheet Fielding vs Lahman Fielding Stats](#Retrosheet-Fielding-vs-Lahman-Fielding-Stats)  

[Retrosheet Pitching (Allowed) vs Retrosheet Hitting](#Retrosheet-Pitching-(allowed)-vs-Retrosheet-Hitting)  
[Lahman Pitching (Allowed) vs Lahman Hitting](#Lahman-Pitching-(allowed)-vs-Lahman-Hitting)  

[Retrosheet Batting vs Retrosheet Team Batting](#Retrosheet-Batting-vs-Retrosheet-Team-Batting)  
[Lahman Batting vs Lahman Team Batting](#Lahman-Batting-vs-Lahman-Team-Batting)  
[Retrosheet Fielding vs Retrosheet Team Fielding](#Retrosheet-Fielding-vs-Retrosheet-Team-Fielding)  
[Lahman Fielding vs Lahman Team Fielding](#Lahman-Fielding-vs-Lahman-Team-Fielding)    
[Retrosheet Pitching vs Retrosheet Team Pitching](#Retrosheet-Pitching-vs-Retrosheet-Team-Pitching)  
[Lahman Pitching vs Lahman Team Pitching](#Lahman-Pitching-vs-Lahman-Team-Pitching)  

## Setup
[Preliminaries](#Preliminaries)  
[Imports and Setup](#Imports-and-Setup)  
[Load Data](#Load-the-Data)   

## Preliminaries

This notebook assumes that the Lahman and Retrosheet data sets have been downloaded and wrangled using the scripts in the `../download_scripts` directory of this repo.

For this notebook, Retrosheet data from 1974 through 2019 inclusive is used.

The `../download_scripts/data_helper.py` function: `from_csv_with_types()` uses pd.read_csv() with dtypes set to the type data read in from: <filename\>_types.csv.  This allows Pandas to use the previously optimized data types to use which require about 1/3rd as much memory.

## MLB Data Summary

Most used csv files.

**Lahman**  
* Stats per Player per Year:
  * batting.csv
  * pitching.csv
  * fielding.csv
* Stats per Team per Year:
  * teams.csv -- contains team_id for both Lahman and Retrosheet
* Other
  * people.csv -- contains player_id for Lahman, Retrosheet and Baseball-Reference
  
**Retrosheet**  
* Stats per Player per Game:
  * batting.csv.gz
  * pitching.csv.gz
  * fielding.csv.gz
* Stats per Team per Game:
  * team_game.csv.gz
* Stats per Game:
  * game.csv.gz 

## Imports and Setup

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import re
from scipy.stats import linregress

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100 # increase dpi, will make figures larger and clearer

In [4]:
import sys

# import data_helper.py from download_scripts directory
sys.path.append('../download_scripts')
import data_helper as dh

In [5]:
data_dir = Path('../data')
lahman_data = data_dir.joinpath('lahman/wrangled').resolve()
retrosheet_data = data_dir.joinpath('retrosheet/wrangled').resolve()

In [6]:
pd.set_option("display.max_columns", 50)

# Load the Data

Loading all the data up front makes the code clearer, but uses more memory.

As optimized Pandas data types were persisted when the data was wrangled, the total memory usage is about 3 times less than if Pandas inferred the data types with pd.read_csv().

Notes:
* every Retrosheet player who appears in a game is in retro_batting, even if they had no plate appearances
* every Retrosheet team appears in retro_team_game
* the Lahman stint value is incremented each time a player is traded in the same year

In [7]:
lahman_people = dh.from_csv_with_types(lahman_data / 'people.csv')
lahman_teams = dh.from_csv_with_types(lahman_data / 'teams.csv')
lahman_batting = dh.from_csv_with_types(lahman_data / 'batting.csv')
lahman_pitching = dh.from_csv_with_types(lahman_data / 'pitching.csv')
lahman_fielding = dh.from_csv_with_types(lahman_data / 'fielding.csv')

In [8]:
# restrict Lahman data to be between 1974 and 2019 inclusive
lahman_batting = lahman_batting.query('1974 <= year <= 2019')
lahman_pitching = lahman_pitching.query('1974 <= year <= 2019')
lahman_fielding = lahman_fielding.query('1974 <= year <= 2019')
lahman_teams = lahman_teams.query('1974 <= year <= 2019')

In [9]:
retro_batting = dh.from_csv_with_types(retrosheet_data / 'batting.csv.gz')
retro_pitching = dh.from_csv_with_types(retrosheet_data / 'pitching.csv.gz')
retro_fielding = dh.from_csv_with_types(retrosheet_data / 'fielding.csv.gz')
retro_team_game = dh.from_csv_with_types(retrosheet_data / 'team_game.csv.gz')
retro_game = dh.from_csv_with_types(retrosheet_data / 'game.csv.gz')

In [10]:
# restrict Retrosheet data to be between 1974 and 2019 inclusive
retro_batting = retro_batting.query('1974 <= game_start_dt.dt.year <= 2019')
retro_pitching = retro_pitching.query('1974 <= game_start_dt.dt.year <= 2019')
retro_fielding = retro_fielding.query('1974 <= game_start_dt.dt.year <= 2019')
retro_team_game = retro_team_game.query('1974 <= game_start_dt.dt.year <= 2019')
retro_game = retro_game.query('1974 <= game_start_dt.dt.year <= 2019')

In [11]:
# verify these years exist in the downloaded data
(retro_batting['game_start_dt'].agg(['min', 'max']).dt.year == (1974, 2019)).all()

True

In [12]:
# verify all the years are in the downloaded data
retro_batting['game_start_dt'].dt.year.nunique() == (2019 - 1974) + 1

True

# Primary and Foreign Key Tests
Perform these tests before checking for data consistency.

In the optional `../download_scripts/postgres_load_data.py` script, primary and foreign key constraints are used to enforce these data checks.

## Lahman People and Lahman/Retrosheet Player IDs
The Lahman people.csv file contains a mapping between Lahman player_id and Retrosheet player_id.

Ensure that this mapping is "one to one" and "onto", which just means:
* for every Lahman player_id there is exactly one Retrosheet player_id
* for every Retrosheet player_id there is exactly one Lahman player_id

Although the code that follows allows for a missing mapping, there are no missing mappings for players between 1974 and 2019 (as of the December Lahman data update).

In [13]:
# verify Lahman player_id is unique
dh.is_unique(lahman_people, ['player_id'])

True

In [14]:
# verify Lahman's retrosheet player_id is unqiue or missing
dh.is_unique(lahman_people, ['retro_id'], ignore_null=True)

True

In [15]:
# verify the mapping is one-to-one and onto using an inner join
retro_players = pd.Series(retro_batting['player_id'].unique(), name='player_id')
print(f'There are {len(retro_players)} unique players in Retrosheet')

There are 9383 unique players in Retrosheet


In [16]:
# use an inner join to verify that the mapping is one-to-one and onto
mapping = lahman_people[['player_id', 'retro_id']].merge(retro_players, how='inner', 
                    left_on=['retro_id'], right_on=['player_id'],
                    suffixes=('_lahman', '_retro'))
print(f'There are {len(mapping)} player_ids that match between Lahman and Retrosheet')

There are 9383 player_ids that match between Lahman and Retrosheet


In [17]:
len(retro_players) == len(mapping)

True

In [18]:
# Given the above, the following must be True
r_players = set(retro_players)
l_players = set(lahman_people['retro_id'])
r_players.issubset(l_players)

True

## Lahman Teams and Lahman/Retrosheet Team IDs
Same analysis as above, but with (team_id, year) instead of player_id.

In [19]:
# verify Lahman (team_id, year) is unique
dh.is_unique(lahman_teams, ['team_id', 'year'])

True

In [20]:
# verify Lahman's retrosheet (team__id, year) is unqiue
dh.is_unique(lahman_teams, ['team_id_retro', 'year'])

True

In [21]:
retro_team_ids = set(zip(retro_team_game['team_id'], 
                         retro_team_game['game_start_dt'].dt.year))
retro_team_ids_df = pd.DataFrame(retro_team_ids, columns=['team_id', 'year'])
print(f'There are {len(retro_team_ids_df)} unique (team_id, year) in Retrosheet')

There are 1288 unique (team_id, year) in Retrosheet


In [22]:
# use an inner join to verify that the mapping is one-to-one and onto
mapping = lahman_teams.merge(retro_team_ids_df, how='inner', 
                             left_on=['team_id_retro', 'year'], 
                             right_on=['team_id','year'])
print(f'There are {len(mapping)} (team_id, year) that match between Lahman and Retrosheet')

There are 1288 (team_id, year) that match between Lahman and Retrosheet


In [23]:
len(retro_team_ids) == len(mapping)

True

In [24]:
# Given the above, the following must be True
lahman_team_ids = set(zip(lahman_teams['team_id_retro'], lahman_teams['year']))
retro_team_ids.issubset(lahman_team_ids)

True

## Lahman Batting/Pitching/Fielding

In [25]:
dh.is_unique(lahman_batting, ['player_id', 'year', 'stint'])

True

In [26]:
dh.is_unique(lahman_pitching, ['player_id', 'year', 'stint'])

True

In [27]:
dh.is_unique(lahman_fielding, ['player_id', 'year', 'stint', 'pos'])

True

## Retrosheet Batting/Pitching/Fielding/Team_Game/Game

In [28]:
# Note: 1 duplicate key was cleaned during wrangling
# The two entries for Chris Young (younc004) were summed into a single entry
dh.game_id_to_url('BOS201708250')
dh.is_unique(retro_batting, ['player_id', 'game_id'])

True

In [29]:
dh.is_unique(retro_pitching, ['player_id', 'game_id'])

True

In [30]:
dh.is_unique(retro_fielding, ['player_id', 'game_id', 'pos'])

True

In [31]:
dh.is_unique(retro_team_game, ['team_id', 'game_id'])

True

In [32]:
dh.is_unique(retro_game, ['game_id'])

True

# Data Consistency Tests
The download scripts ensured that columns with the same meaning, were given the same name.  This makes it easy to compare data between CSV files using their column names.

## Retrosheet Batting vs Lahman Batting Players

In [33]:
# TODO add this to pytest tests
# verify that Lahman and Retrosheet have stats on the same set of batters
lahman_batters = pd.merge(lahman_batting['player_id'], lahman_people[['player_id', 'retro_id']])
r_batters = set(retro_batting['player_id'].unique())
l_batters = set(lahman_batters['retro_id'].unique())
r_batters == l_batters

True

## Retrosheet Batting vs Lahman Batting Stats

In [34]:
# batting columns to compare
cols = set(retro_batting.columns) & set(lahman_batting.columns)
cols -= {'player_id', 'team_id'}
len(cols)

17

In [35]:
cols

{'ab',
 'bb',
 'cs',
 'double',
 'g',
 'gidp',
 'h',
 'hbp',
 'hr',
 'ibb',
 'r',
 'rbi',
 'sb',
 'sf',
 'sh',
 'so',
 'triple'}

In [36]:
# aggregate the stats in common for all players for all years (1974 thru 2019)
l = lahman_batting[cols]
r = retro_batting[cols]

l_sums = l.agg('sum')
l_sums.sort_index(inplace=True)

r_sums = r.agg('sum')
r_sums.sort_index(inplace=True)

In [37]:
# compute the relative differences
np.abs(1.0 - (l_sums / r_sums))

ab        1.426074e-07
bb        1.487730e-06
cs        3.295273e-05
double    2.922456e-06
g         1.397775e-06
gidp      8.906306e-05
h         0.000000e+00
hbp       0.000000e+00
hr        0.000000e+00
ibb       1.776294e-05
r         0.000000e+00
rbi       5.724236e-06
sb        2.964764e-05
sf        0.000000e+00
sh        1.411652e-05
so        0.000000e+00
triple    0.000000e+00
dtype: float64

In [38]:
# find the largest relative difference
print(f'{np.abs(1.0 - (l_sums / r_sums)).max():8.6f}')

0.000089


In [39]:
# all 17 batting attributes from 1974-2019 between Lahman and Retrosheet
# are within plus/minus 0.01% of each other when summed
(np.abs(1.0 - (l_sums / r_sums)) < .0001).all()

True

## Retrosheet Pitching vs Lahman Pitching Players

In [40]:
# TODO add this to pytest tests
# verify that Lahman and Retrosheet have stats on exactly the same set of pitchers
lahman_pitchers = pd.merge(lahman_pitching['player_id'], lahman_people[['player_id', 'retro_id']])
r_pitchers = set(retro_pitching['player_id'].unique())
l_pitchers = set(lahman_pitchers['retro_id'].unique())
r_pitchers == l_pitchers

True

## Retrosheet Pitching vs Lahman Pitching Stats

In [41]:
# pitching columns to compare
cols = set(retro_pitching.columns) & set(lahman_pitching.columns)
cols -= {'player_id', 'team_id', 'year'}
len(cols)

21

In [42]:
cols

{'bb',
 'bk',
 'cg',
 'er',
 'g',
 'gf',
 'gidp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ibb',
 'l',
 'r',
 'sf',
 'sh',
 'sho',
 'so',
 'sv',
 'w',
 'wp'}

In [43]:
# aggregate the stats in common for all players for all years (1974 thru 2019)
l = lahman_pitching[cols]
r = retro_pitching[cols]

l_sums = l.agg('sum')
l_sums.sort_index(inplace=True)

r_sums = r.agg('sum')
r_sums.sort_index(inplace=True)

In [44]:
# compute the relative differences
np.abs(1.0 - (l_sums / r_sums))

bb      0.000001
bk      0.000403
cg      0.000327
er      0.000002
g       0.000105
gf      0.000032
gidp    0.000006
gs      0.000000
h       0.000000
hbp     0.000000
hr      0.000005
ibb     0.000018
l       0.000000
r       0.000000
sf      0.000000
sh      0.000014
sho     0.000598
so      0.000000
sv      0.000081
w       0.000000
wp      0.000355
dtype: float64

In [45]:
# find the largest relative difference
print(f'{np.abs(1.0 - (l_sums / r_sums)).max():8.6f}')

0.000598


In [46]:
# verify all values between 1974 and 2019 are within plus/minus 0.06% of each other
(np.abs(1.0 - (l_sums / r_sums)) < .0006).all()

True

## Retrosheet Fielding vs Lahman Fielding Players

In [47]:
# TODO add this to pytest tests
# verify that Lahman and Retrosheet have stats on exactly the same set of fielders
lahman_fielders = pd.merge(lahman_fielding['player_id'], lahman_people[['player_id', 'retro_id']])
r_fielders = set(retro_fielding['player_id'].unique())
l_fielders = set(lahman_fielders['retro_id'].unique())
r_fielders == l_fielders

False

In [48]:
l_fielders - r_fielders

set()

In [49]:
r_fielders - l_fielders

{'olivt102'}

In [50]:
# Retrosheet has a fielder not in Lahman, what are the fielding stats for this player?
retro_fielding.query('player_id == "olivt102"')

,game_id,player_id,pos,team_id,g,gs,inn_outs,tc,po,a,e,dp,tp,pb,xi,game_start_dt
791764,BOS197604190,olivt102,2B,MIN,0,1,0,0,0,0,0,0,0,0,0,1976-04-19 11:05:00
791786,BOS197604200,olivt102,2B,MIN,0,1,0,0,0,0,0,0,0,0,0,1976-04-20 15:03:00
816021,NYA197604180,olivt102,2B,MIN,0,1,0,0,0,0,0,0,0,0,0,1976-04-18 14:07:00
831226,TEX197604110,olivt102,2B,MIN,0,1,0,0,0,0,0,0,0,0,0,1976-04-11 00:00:00


Tony Oliva (olivt102) started 4 games as a second baseman.  
He had zero total chances (tc) and no outs were recorded during the time he played second base.  

This sounds like a pinch hitter or a designated hitter, rather than a fielder.  Let's check the box scores and player information on Baseball-Reference.

In [51]:
# right click on each generated link and open in a new tab
dh.game_id_to_url('BOS197604190')
dh.game_id_to_url('BOS197604200')
dh.game_id_to_url('NYA197604180')
dh.game_id_to_url('TEX197604110')

bb_player_id = lahman_people.query('retro_id == "olivt102"')['bb_ref_id'].values[0]
dh.player_id_to_url(bb_player_id)

Tony Oliva lead off in each of these four away games and was immediately replaced, whether he got on base or not.  He was only in the game to hit once.  He might have been unable to run or field well for these games, but he could still hit, so he was in the lineup.

Tony was listed as the starting second baseman, even though he was a career right fielder.  Tony was not going to play second base, the guy who replaced him, Jerry Terrell, was.

The cwdaily parser created a fielding record for Tony as he was on the lineup card as the starting second baseman.

It would be hard to argue that either the Lahman or Retrosheet data is wrong in this scenario.  The only difference is that Retrosheet show 4 starts as a second baseman for Tony Oliva that Lahman does not.

The fielding players are effectively the same in Lahman and Retrosheet (1974 through 2019).

## Retrosheet Fielding vs Lahman Fielding Stats

In [52]:
# fielding columns to compare
cols = set(retro_fielding.columns) & set(lahman_fielding.columns)
cols -= {'player_id', 'team_id', 'year'}
len(cols)

8

In [53]:
cols

{'a', 'dp', 'e', 'g', 'gs', 'inn_outs', 'po', 'pos'}

In [54]:
# aggregate the stats in common per position over all players over all years (1974 thru 2019)
l = lahman_fielding[cols]
r = retro_fielding[cols]

l_sums = l.groupby('pos').agg('sum')
l_sums.sort_index(inplace=True)

r_sums = r.groupby('pos').agg('sum')
r_sums.sort_index(inplace=True)

In [55]:
# compute the relative differences
np.abs(1.0 - (l_sums / r_sums))

,po,dp,inn_outs,g,a,gs,e
pos,,,,,,,
1B,0.000482,0.000737,2.470157e-05,0.000008,0.006393,0.000029,0.000833
2B,0.002122,0.000007,2.179551e-06,0.000017,0.001008,0.000063,0.000309
3B,0.001850,0.000908,9.807978e-06,0.000177,0.001483,0.000019,0.000109
C,0.000668,0.000982,5.448877e-07,0.000059,0.007810,0.000005,0.000683
CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P,0.005532,0.002302,4.722360e-06,0.000105,0.001890,0.000000,0.001415
RF,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# Lahman uses OF for sum of LF, CF, RF -- account for this
r_sums.loc['OF'] = r_sums.loc['LF'] + r_sums.loc['CF'] + r_sums.loc['RF']
r_sums = r_sums.drop(['LF', 'CF', 'RF'])
r_sums.sort_index(inplace=True)
r_sums

,po,dp,inn_outs,g,a,gs,e
pos,,,,,,,
1B,1798401.0,171005.0,5505722.0,237097.0,138119.0,205280.0,14409.0
2B,422753.0,135428.0,5505722.0,235233.0,599404.0,205280.0,19443.0
3B,149770.0,37465.0,5505722.0,237078.0,404527.0,205280.0,27552.0
C,1349560.0,13236.0,5505722.0,236247.0,106279.0,205280.0,14642.0
OF,1352944.0,8256.0,16517166.0,735430.0,38885.0,615840.0,24381.0
P,112264.0,17807.0,5505722.0,695259.0,243922.0,205280.0,16956.0
SS,319687.0,127447.0,5505722.0,232003.0,610365.0,205280.0,29333.0


In [57]:
# The above will overcount games, as Lahman will have a player in the game once as an OF, 
# whereas Retrosheet may have him in the game as both a LF and CF, for example.
# filt = retro_fielding['pos'].isin(['LF', 'CF', 'RF'])
# r_of = retro_fielding[filt]
r_of = retro_fielding.query('pos in ["LF", "CF", "RF"]')

total_dups = r_of.duplicated(subset=['player_id', 'game_id'], keep=False).sum()
counted_dups = r_of.duplicated(subset=['player_id', 'game_id'], keep='first').sum()

r_sums.loc['OF', 'g'] -= (total_dups - counted_dups)
r_sums

,po,dp,inn_outs,g,a,gs,e
pos,,,,,,,
1B,1798401.0,171005.0,5505722.0,237097.0,138119.0,205280.0,14409.0
2B,422753.0,135428.0,5505722.0,235233.0,599404.0,205280.0,19443.0
3B,149770.0,37465.0,5505722.0,237078.0,404527.0,205280.0,27552.0
C,1349560.0,13236.0,5505722.0,236247.0,106279.0,205280.0,14642.0
OF,1352944.0,8256.0,16517166.0,711605.0,38885.0,615840.0,24381.0
P,112264.0,17807.0,5505722.0,695259.0,243922.0,205280.0,16956.0
SS,319687.0,127447.0,5505722.0,232003.0,610365.0,205280.0,29333.0


In [58]:
# compute relative differences
np.abs(1.0 - l_sums / r_sums)

,po,dp,inn_outs,g,a,gs,e
pos,,,,,,,
1B,0.000482,0.000737,2.470157e-05,0.000008,0.006393,0.000029,0.000833
2B,0.002122,0.000007,2.179551e-06,0.000017,0.001008,0.000063,0.000309
3B,0.001850,0.000908,9.807978e-06,0.000177,0.001483,0.000019,0.000109
C,0.000668,0.000982,5.448877e-07,0.000059,0.007810,0.000005,0.000683
OF,0.000072,0.001453,6.599195e-06,0.000311,0.002546,0.000013,0.000656
P,0.005532,0.002302,4.722360e-06,0.000105,0.001890,0.000000,0.001415
SS,0.001301,0.000730,8.354944e-06,0.000004,0.001009,0.000068,0.000102


In [59]:
# find the largest relative difference
print(f'{np.abs(1.0 - (l_sums / r_sums)).max().max():8.6f}')

0.007810


In [60]:
# verify all values between 1974 and 2019 are within plus/minus 0.8% of each other
np.abs(1.0 - (l_sums / r_sums)).max().max() < 0.008

True

# Retrosheet Pitching (allowed) vs Retrosheet Hitting

In [61]:
#TODO add this to pytest
exclude = ['game_id', 'team_id', 'player_id', 'g', 'game_start_dt']
cols = set(retro_pitching.columns) & set(retro_batting.columns) - set(exclude)
cols = list(cols)
cols

['r',
 'hr',
 'ab',
 'tb',
 'sf',
 'ibb',
 'sh',
 'xi',
 'hbp',
 'double',
 'h',
 'hr4',
 'gidp',
 'triple',
 'so',
 'bb']

In [62]:
# sum over all pitchers over all years
p = retro_pitching[cols].agg('sum')
p

r          923811
hr         195985
ab        7012260
tb        2840020
sf          58275
ibb         56297
sh          70839
xi           1060
hbp         58097
double     342178
h         1826823
hr4          4710
gidp       157192
triple      41532
so        1301700
bb         672165
dtype: int64

In [63]:
# sum over all batters over all years
b = retro_batting[cols].agg('sum')
b

r          923811
hr         195985
ab        7012260
tb        2840020
sf          58275
ibb         56297
sh          70839
xi           1060
hbp         58097
double     342178
h         1826823
hr4          4710
gidp       157192
triple      41532
so        1301700
bb         672165
dtype: int64

In [64]:
# Retrosheet is completely consistent
p.equals(b)

True

# Lahman Pitching (allowed) vs Lahman Hitting

In [65]:
#TODO add this to pytest
exclude = ['lg_id', 'player_id', 'stint', 'team_id', 'year', 'g']
pkey = ['player_id', 'stint', 'year']
cols = set(lahman_pitching.columns) & set(lahman_batting.columns)
cols -= set(exclude)
cols

{'bb', 'gidp', 'h', 'hbp', 'hr', 'ibb', 'r', 'sf', 'sh', 'so'}

In [66]:
# sum over all pitchers over all years (1974 thru 2019)
p = lahman_pitching[cols].agg('sum')
p

r        923811.0
hr       195986.0
sf        58275.0
ibb       56296.0
sh        70838.0
hbp       58097.0
so      1301700.0
h       1826823.0
gidp     157193.0
bb       672166.0
dtype: float64

In [67]:
# sum over all batters over all years (1974 thru 2019)
b = lahman_batting[cols].agg('sum')
b

r        923811.0
hr       195985.0
sf        58275.0
ibb       56296.0
sh        70838.0
hbp       58097.0
so      1301700.0
h       1826823.0
gidp     157178.0
bb       672166.0
dtype: float64

In [68]:
# compute the relative differences
np.abs(1.0 - p/b)

r       0.000000
hr      0.000005
sf      0.000000
ibb     0.000000
sh      0.000000
hbp     0.000000
so      0.000000
h       0.000000
gidp    0.000095
bb      0.000000
dtype: float64

In [69]:
# find the max difference
np.abs(1.0 - p/b).max()

9.543320311999892e-05

In [70]:
# the biggest difference is less than 0.01%
(np.abs(1.0 - p/b).max() < 0.0001).all()

True

# Retrosheet Batting vs Retrosheet Team Batting

In [71]:
exclude = ['game_id', 'team_id', 'player_id', 'game_start_dt']
cols = set(retro_batting.columns) & set(retro_team_game.columns) - set(exclude)
cols = list(cols)
len(cols)

17

In [72]:
cols

['r',
 'hr',
 'sf',
 'ab',
 'ibb',
 'xi',
 'sh',
 'sb',
 'cs',
 'hbp',
 'double',
 'h',
 'gidp',
 'triple',
 'rbi',
 'so',
 'bb']

In [73]:
# retro_batting was wrangled from the output of cwdaily
b = retro_batting[['game_id', 'team_id'] + cols].groupby(['game_id', 'team_id']).agg('sum')
b = b.reset_index()
b.head(4)

,game_id,team_id,r,hr,sf,ab,ibb,xi,sh,sb,cs,hbp,double,h,gidp,triple,rbi,so,bb
0,ANA199704020,ANA,5,1,1,38,0,0,0,2,0,0,0,12,0,0,5,7,5
1,ANA199704020,BOS,6,1,0,36,0,0,0,0,0,1,2,9,0,0,6,12,8
2,ANA199704030,ANA,2,0,1,27,1,0,0,0,0,0,2,7,2,0,2,1,4
3,ANA199704030,BOS,0,0,0,29,0,0,0,0,0,0,0,5,3,0,0,5,0


In [74]:
# retro_team_game was wrangled from the output of cwgame
tg = retro_team_game[['game_id', 'team_id'] + cols].sort_values(['game_id', 'team_id']).reset_index(drop=True)
tg.head(4)

,game_id,team_id,r,hr,sf,ab,ibb,xi,sh,sb,cs,hbp,double,h,gidp,triple,rbi,so,bb
0,ANA199704020,ANA,5,1,1,38,0,0,0,2,0,0,0,12,0,0,5,7,5
1,ANA199704020,BOS,6,1,0,36,0,0,0,0,0,1,2,9,0,0,6,12,8
2,ANA199704030,ANA,2,0,1,27,1,0,0,0,0,0,2,7,2,0,2,1,4
3,ANA199704030,BOS,0,0,0,29,0,0,0,0,0,0,0,5,3,0,0,5,0


In [75]:
# Retrosheet is completely consistent between cwdaily and cwgame for batting
b.equals(tg)

True

# Lahman Batting vs Lahman Team Batting

In [76]:
#TODO add this to pytest
exclude = ['lg_id', 'team_id', 'year', 'g']
key = ['team_id', 'year']
cols = set(lahman_batting.columns) & set(lahman_teams.columns) - set(exclude)
cols = list(cols)
len(cols)

12

In [77]:
cols

['r', 'hr', 'sf', 'ab', 'sb', 'cs', 'hbp', 'double', 'h', 'triple', 'so', 'bb']

In [78]:
b = lahman_batting[key + cols].groupby(key).agg('sum')
b = b.reset_index()
b.head(4)

,team_id,year,r,hr,sf,ab,sb,cs,hbp,double,h,triple,so,bb
0,ANA,1997,829,161,57,5628,126,72,45,279,1531,25,953,617
1,ANA,1998,787,147,41,5630,93,45,48,314,1530,27,1028,510
2,ANA,1999,711,158,42,5494,71,45,43,248,1404,22,1022,511
3,ANA,2000,864,236,43,5628,93,52,47,309,1574,34,1024,608


In [79]:
t = lahman_teams[key + cols].sort_values(key).reset_index(drop=True)
t.head(4)

,team_id,year,r,hr,sf,ab,sb,cs,hbp,double,h,triple,so,bb
0,ANA,1997,829,161,57,5628,126,72,45,279,1531,25,953,617
1,ANA,1998,787,147,41,5630,93,45,48,314,1530,27,1028,510
2,ANA,1999,711,158,42,5494,71,45,43,248,1404,22,1022,511
3,ANA,2000,864,236,43,5628,93,52,47,309,1574,34,1024,608


In [80]:
# compute the relative differences
np.abs(1.0 - b[cols]/t[cols])

,r,hr,sf,ab,sb,cs,hbp,double,h,triple,so,bb
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
# find the max difference
np.abs(1.0 - b[cols]/t[cols]).max()

r         0.0
hr        0.0
sf        0.0
ab        0.0
sb        0.0
cs        0.0
hbp       0.0
double    0.0
h         0.0
triple    0.0
so        0.0
bb        0.0
dtype: float64

In [82]:
# the biggest difference is zero
np.abs(1.0 - b[cols]/t[cols]).max().max() == 0.0

True

# Retrosheet Fielding vs Retrosheet Team Fielding

In [83]:
cols = ['a', 'e', 'po', 'pb']
cols

['a', 'e', 'po', 'pb']

In [84]:
# retro_fielding was wrangled from the output of cwdaily
f = retro_fielding[['game_id', 'team_id'] + cols].groupby(['game_id', 'team_id']).agg('sum')
f = f.reset_index()
f.head(4)

,game_id,team_id,a,e,po,pb
0,ANA199704020,ANA,6,1,27,0
1,ANA199704020,BOS,8,1,27,0
2,ANA199704030,ANA,13,0,27,0
3,ANA199704030,BOS,9,0,24,0


In [85]:
# retro_team_game was wrangled from the output of cwgame
tg = retro_team_game[['game_id', 'team_id'] + cols].sort_values(
    ['game_id', 'team_id']).reset_index(drop=True)
tg.head(4)

,game_id,team_id,a,e,po,pb
0,ANA199704020,ANA,6,1,27,0
1,ANA199704020,BOS,8,1,27,0
2,ANA199704030,ANA,13,0,27,0
3,ANA199704030,BOS,9,0,24,0


In [86]:
# Retrosheet is completely consistent between cwdaily and cwgame for fielding
f.equals(tg)

True

# Lahman Fielding vs Lahman Team Fielding

In [87]:
#TODO add this to pytest
# dp is excluded because in fielding, each fielder involved gets a dp
# whereas in team only one dp is counted
exclude = ['lg_id', 'team_id', 'year', 'g', 'dp', 'player_id']
key = ['team_id', 'year']
cols = set(lahman_fielding.columns) & set(lahman_teams.columns) - set(exclude)
cols = list(cols)
len(cols)

1

In [88]:
cols

['e']

In [89]:
f = lahman_fielding[key + cols].groupby(key).agg('sum')
f = f.sort_index().reset_index()
f.head(4)

,team_id,year,e
0,ANA,1997,123
1,ANA,1998,106
2,ANA,1999,106
3,ANA,2000,134


In [90]:
t = lahman_teams[key+cols]
t = t.sort_values(key).reset_index(drop=True)
t.head(4)

,team_id,year,e
0,ANA,1997,123
1,ANA,1998,106
2,ANA,1999,106
3,ANA,2000,134


In [91]:
m = pd.merge(f, t, 
             left_on=['team_id', 'year'],
             right_on=['team_id', 'year'],
             suffixes=['_fielding', '_teams'])
m.head()

,team_id,year,e_fielding,e_teams
0,ANA,1997,123,123
1,ANA,1998,106,106
2,ANA,1999,106,106
3,ANA,2000,134,134
4,ANA,2001,103,103


In [92]:
# query() needs the dtypes to be the same
m['e_fielding'] = m['e_fielding'].astype('int')
m['e_teams'] = m['e_teams'].astype('int')
m.query("e_fielding != e_teams")

,team_id,year,e_fielding,e_teams
653,MIN,1977,144,143
1014,SDN,2007,94,92


When comparing large numbers, it is best to examine their relative differences.
When comparing small numbers, it is best to look at their absolute differences

In [93]:
(f[cols]-t[cols]).max()

e    2.0
dtype: float64

In [94]:
((f[cols]  - t[cols]).max() <= 2).all()

True

# Retrosheet Pitching vs Retrosheet Team Pitching

In [95]:
cols = ['wp', 'bk', 'er']
cols

['wp', 'bk', 'er']

In [96]:
# retro_pitching was wrangled from the output of cwdaily
p = retro_pitching[cols].agg('sum')
p

wp     64799
bk      9919
er    841197
dtype: int64

In [97]:
# retro_team_game was wrangled from the output of cwgame
tg = retro_team_game[cols].agg('sum')
tg

wp     64799
bk      9919
er    841197
dtype: int64

In [98]:
# Retrosheet is completely consistent between cwdaily and cwgame for fielding
p.equals(tg)

True

# Lahman Pitching vs Lahman Team Pitching

In [99]:
#TODO add this to pytest
# most of the common columns are for batting, not pitching
# era cannot be summed
# sho for team is counted differently than for pitcher
# er for team is counted differently than for pitcher
exclude = ['lg_id', 'team_id', 'year', 'g', 'era',
          'bb', 'h', 'hbp', 'hr', 'r', 'sf', 'so', 'sho', 'er']
key = ['team_id', 'year']
cols = set(lahman_pitching.columns) & set(lahman_teams.columns) - set(exclude)
cols = list(cols)
len(cols)

5

In [100]:
cols

['sv', 'l', 'ip_outs', 'w', 'cg']

In [101]:
p = lahman_pitching[key + cols].groupby(key).agg('sum')
p = p.sort_index().reset_index()
p.head(4)

,team_id,year,sv,l,ip_outs,w,cg
0,ANA,1997,39,78,4364,84,9
1,ANA,1998,52,77,4332,85,3
2,ANA,1999,37,92,4294,70,4
3,ANA,2000,46,80,4344,82,5


In [102]:
t = lahman_teams[key+cols]
t = t.sort_values(key).reset_index(drop=True)
t.head(4)

,team_id,year,sv,l,ip_outs,w,cg
0,ANA,1997,39,78,4364,84,9
1,ANA,1998,52,77,4332,85,3
2,ANA,1999,37,92,4294,70,4
3,ANA,2000,46,80,4344,82,5


In [103]:
# dtypes need to be the same
for col in p.columns:
    if not col == 'year' and not col == 'team_id':
        p[col] = p[col].astype('int')
        t[col] = t[col].astype('int')

In [104]:
np.abs(p[col] - t[col]).max()

0

In [105]:
m = pd.merge(p, t, on = ['team_id','year'], suffixes=['_p', '_t'])
s_cols = m.columns.sort_values()
m = m[s_cols]

In [106]:
# display the differences
for col in cols:
    cp = col+'_p'
    ct = col+'_t'
    query = cp + ' != '+ ct
    tmp = m.query(query)[['team_id', 'year'] +[cp, ct]]
    if len(tmp):
        display(tmp)

,team_id,year,l_p,l_t
196,CHA,1979,86,87
329,CLE,1974,84,85


,team_id,year,w_p,w_t
407,DET,1979,84,85
1184,TEX,1974,83,84


Twice, the team value for wins and losses was 1 greater than the sum of the pitchers wins and losses for that team.

## Summary
Real world data is often messy and inaccurate.  It may seem like a lot of work to cross check the data in different ways, but it is necessary to do so before performing any data analysis, otherwise the data analysis could be meaningless.